Proyecto: Predicción de Lluvia - Colombia

# LIMPIEZA, CARGA Y UNIFICACIÓN DE ARCHIVOS

## Librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import os
import plotly.express as pex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.colors as colors
from scipy.stats import mode
from scipy.stats import tstd
from statistics import multimode

*Sistema de archivos*

    ./
    
    ./Proyecto/
        ./Proyecto/Proyecto_bootcamp_cuaderno.ipynb

    ./Datasets/
        ./Datasets/...... .csv
    
    ./CleanDatasets/
        ./CleanDatasets/.....  .parquet


En la siguiente celda imprimimos la cantidad de valores diferentes por cada columna con el objetivo de identificar incoherencias en los datos y posibles valores duplicados.
Por ejemplo, es posible que un municipio como Bogotá se encuentre escrito de diferentes formas como: 'Bogota', 'Bogotá, D.C.' ...etc pero que corresponden al mismo municipio. Esto puede suceder con los nombres de las estaciones y con demás columnas con formato category

In [ ]:
df0=pd.read_parquet("../temp/Temp_0.parquet")
df1=pd.read_parquet("../temp/Temp_1.parquet")
df2=pd.read_parquet("../temp/Temp_2.parquet")
df3=pd.read_parquet("../temp/Temp_3.parquet")
df4=pd.read_parquet("../temp/Temp_4.parquet")

In [5]:
df_part=pd.concat([df0,df1,df2,df3,df4],ignore_index=True)

In [6]:
df_part

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0048015050,0071,2024-01-04 21:58:00,21.100000,AEROPUERTO VASQUEZ COBO,AMAZONAS,LETICIA,AMAZONAS - DIRECTOS,-4.19386111,-69.94091667,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
1,0015065180,0071,2024-01-04 19:46:00,22.299999,AEROPUERTO ALM. PADILLA -,LA GUAJIRA,RIOHACHA,CARIBE - GUAJIRA,11.5284444,-72.91772222,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
2,3502500135,0071,2024-01-04 03:56:00,17.000000,GUAYABETAL POLLO OLIMPICO,CUNDINAMARCA,GUAYABETAL,META,4.22553,-73.81481,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
3,0016015501,0071,2024-01-04 16:30:00,28.500000,AEROPUERTO CAMILO DAZA,NORTE DE SANTANDER,CÚCUTA,CATATUMBO,7.93028,-72.50917,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
4,0026125710,0071,2024-01-04 06:58:00,20.900000,AEROPUERTO MATECANA,RISARALDA,PEREIRA,CAUCA,4.812675,-75.73951944,GPRS - TEMPERATURA DEL AIRE A 2 m,°C
...,...,...,...,...,...,...,...,...,...,...,...,...
78017982,0052025501,0068,2019-03-15 07:40:00,16.380951,LA VEGA - AUT,CAUCA,LA VEGA,PATÍA,2.092941667,-76.81202778,Temp Aire 2 m,°C
78017983,0035075080,0068,2015-07-02 12:00:00,8.500000,PARAMO RABANAL - AUT,BOYACÁ,VENTAQUEMADA,META,5.392388889,-73.56277778,Temp Aire 2 m,°C
78017984,0052055170,0068,2018-01-09 12:00:00,22.700001,LA JOSEFINA - AUT,NARIÑO,CONTADERO,PATÍA,0.930305556,-77.49119444,Temp Aire 2 m,°C
78017985,0027015330,0068,2017-03-26 11:00:00,21.700001,APTO OLAYA HERRERA - TX GPRS,ANTIOQUIA,MEDELLÍN,NECHÍ,6.221,-75.589,Temp Aire 2 m,°C


In [7]:
df_part.UnidadMedida.unique()

['°C']
Categories (1, object): ['°C']

In [8]:
df_part['Latitud'] = df_part['Latitud'].astype(np.float64)
df_part['Longitud'] = df_part['Longitud'].astype(np.float64)
df_part['CodigoEstacion'] = df_part['CodigoEstacion'].astype(str)
df_part['Departamento'] = df_part['Departamento'].astype(str)
df_part['Municipio'] = df_part['Municipio'].astype(str)

In [9]:
df_part=df_part[['CodigoEstacion','Departamento','Municipio','Latitud','Longitud','FechaObservacion','ValorObservado']]
df_part_aux=df_part.groupby(['CodigoEstacion','Departamento','Municipio','Latitud','Longitud'], observed=True).agg(TotalEntries=('ValorObservado','count')).reset_index()
df_part=df_part_aux.copy()

In [10]:
df_part = df_part.rename({'CodigoEstacion':'CodSta',
                            'Departamento':'Dept',
                            'Municipio':'City',
                            'Latitud':'Lat',
                            'Longitud':'Long',
                            'FechaObservacion':'Date',
                            'ValorObservado':'Temp'
                            }, 
                            axis=1)

In [11]:
df_part.nunique()

CodSta          602
Dept             45
City            412
Lat             842
Long            748
TotalEntries    813
dtype: int64

In [12]:
df_part.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 877 entries, 0 to 876
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CodSta        877 non-null    object 
 1   Dept          877 non-null    object 
 2   City          877 non-null    object 
 3   Lat           877 non-null    float64
 4   Long          877 non-null    float64
 5   TotalEntries  877 non-null    int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 41.2+ KB


Note que existen 403 códigos de estación pero existen 626 nombres de estación, 1191 latitudes y 1098 longitudes. Esto indica que para algunos nombres de estación y coordenadas están escritos de manera diferente para una misma estación. Vamos a hacer un pequeño análisis para identificar estas incongruencias y evitar pérdidas de información.

In [13]:
df_part.groupby(['CodSta','Lat','Long'], observed=True).agg({'TotalEntries':'sum'}).reset_index()

,CodSta,Lat,Long,TotalEntries
0,0011025501,5.888719,-76.145167,501365
1,0011030010,5.375000,-76.613000,35962
2,0011030010,5.380000,-76.610000,4404
3,0011035030,5.284828,-76.627822,3128
4,0011035030,5.285000,-76.628000,17607
...,...,...,...,...
863,5202500132,1.814142,-76.782833,4878
864,5205500123,1.054806,-77.270250,12631
865,5311500056,3.848308,-76.987017,21572
866,5311500121,3.645194,-76.564750,25296


Se observa que al filtrar por Código de Estación, Nombre de Estación, Latitud y Longitud se encuentran 1375 filas lo cual contrasta con los 894 valores únicos de CodigoEstacion, se evidencia que algunas estaciones tienen el nombre de la estación o las coordenadas escritas de maneras diferentes. Empezaremos por corregir las coordenadas Latitud y Longitud:

##### Coordenadas

Aplicamos una primera aproximación donde redondeamos las coordenadas *Latitud* y *Longitud* a 3 decimales, esto se justifica en que 0.001° equivalen aproximadamente a 100m. Realizamos un conteó de los registros totales que debe coincidir con el valor inicial de *199781152*.

En esta primera celda calculamos el total de registros por cada ubicación, incluimos el nombre de la estación para consultas futuras en el procedimiento de limpieza de las coordenadas.

In [14]:
coordSta_0 = df_part.groupby(['CodSta','Lat','Long'], observed=True).agg({'TotalEntries':'sum'}).reset_index()
coordSta_0[['Lat','Long']] = coordSta_0[['Lat','Long']].round(3)
print(f'Total Entries: {coordSta_0['TotalEntries'].agg('sum')}')
coordSta_0

Total Entries: 78017987


,CodSta,Lat,Long,TotalEntries
0,0011025501,5.889,-76.145,501365
1,0011030010,5.375,-76.613,35962
2,0011030010,5.380,-76.610,4404
3,0011035030,5.285,-76.628,3128
4,0011035030,5.285,-76.628,17607
...,...,...,...,...
863,5202500132,1.814,-76.783,4878
864,5205500123,1.055,-77.270,12631
865,5311500056,3.848,-76.987,21572
866,5311500121,3.645,-76.565,25296


Al agrupar el dataframe por CodSta, Lat y Long se obtiene un total de 968 filas, de manera que existen Códigos de estación con diferentes coordenadas. Realizaremos ajustes a las coordenadas para obtener una coordenada única para cada estación. 

In [15]:
coordSta_1 = coordSta_0.groupby(['CodSta','Lat','Long'],observed=True).agg({'TotalEntries':'sum'}).reset_index()
print(f'Total Entries: {coordSta_1['TotalEntries'].agg('sum')}')
coordSta_1

Total Entries: 78017987


,CodSta,Lat,Long,TotalEntries
0,0011025501,5.889,-76.145,501365
1,0011030010,5.375,-76.613,35962
2,0011030010,5.380,-76.610,4404
3,0011035030,5.285,-76.628,20735
4,0011045010,5.691,-76.644,43970
...,...,...,...,...
642,5202500132,1.814,-76.783,4878
643,5205500123,1.055,-77.270,12631
644,5311500056,3.848,-76.987,21572
645,5311500121,3.645,-76.565,25296


Definimos una función que nos permite encontrar la distancia en Km entre dos ubicaciones en base a las coordenadas.

In [16]:
def Distance(Lat1:float,Long1:float,Lat2:float,Long2:float):
    Lat1, Long1, Lat2, Long2 = map(np.radians, [Lat1, Long1, Lat2, Long2])
    DeltaLong = Long2-Long1
    DeltaLat = Lat2-Lat1
    R = 6371   #Asumiendo un modelo esférico de la tierra  
    return round(2*R*np.arcsin(np.sqrt(np.sin(DeltaLat/2)**2 + ( np.cos(Lat1)*np.cos(Lat2)*np.sin(DeltaLong/2)**2 ))),3)

Calculamos para cada Código de Estación el valor medio de las diferentes coordenadas. Adicionalmente calculamos la columna *Distance* como la distancia de la coordenada registrada al punto coordenado medio.

In [17]:
coordSta_aux = coordSta_1.groupby(['CodSta'], observed=True).agg({'Lat':'mean','Long':'mean'}).round(3)
coordSta_aux = coordSta_aux.reset_index()
coordSta_1 = coordSta_1[['CodSta','Lat','Long','TotalEntries']].merge(coordSta_aux, how='inner', on='CodSta', suffixes=('','_mean'))
coordSta_1['Distance'] = coordSta_1.apply(lambda x: Distance(x['Lat'],x['Long'],x['Lat_mean'],x['Long_mean']), axis=1)
print(f'Total Entries: {coordSta_1['TotalEntries'].agg('sum')}')
coordSta_1

Total Entries: 78017987


,CodSta,Lat,Long,TotalEntries,Lat_mean,Long_mean,Distance
0,0011025501,5.889,-76.145,501365,5.889,-76.145,0.000
1,0011030010,5.375,-76.613,35962,5.377,-76.612,0.248
2,0011030010,5.380,-76.610,4404,5.377,-76.612,0.400
3,0011035030,5.285,-76.628,20735,5.285,-76.628,0.000
4,0011045010,5.691,-76.644,43970,5.691,-76.644,0.000
...,...,...,...,...,...,...,...
642,5202500132,1.814,-76.783,4878,1.814,-76.783,0.000
643,5205500123,1.055,-77.270,12631,1.055,-77.270,0.000
644,5311500056,3.848,-76.987,21572,3.848,-76.987,0.000
645,5311500121,3.645,-76.565,25296,3.645,-76.565,0.000


Consideramos un radio de precisión de las coordenadas de 1.5Km. Los registros de coordenadas con distancias menores a 1.5Km los reemplazamos con el valor medio de las coordenadas. El dataframe *coordSta_2* almacenará las coordenadas de las estaciones que presentan variaciones en sus coordenadas, menores a un radio de 1.5Km desde el punto medio de las coordenadas.

In [18]:
coordSta_2 = coordSta_1[coordSta_1['Distance']<=1.5].copy()
coordSta_2['Lat'] = coordSta_2['Lat_mean']
coordSta_2['Long'] = coordSta_2['Long_mean']
coordSta_2 = coordSta_2.drop(['Lat_mean','Long_mean'], axis=1)
coordSta_2 = coordSta_2.groupby(['CodSta','Lat','Long']).agg({'TotalEntries':'sum'})
coordSta_2 = coordSta_2.reset_index()
print(f'Total Entries: {coordSta_2['TotalEntries'].agg('sum')}')
coordSta_2

Total Entries: 77300914


,CodSta,Lat,Long,TotalEntries
0,0011025501,5.889,-76.145,501365
1,0011030010,5.377,-76.612,40366
2,0011035030,5.285,-76.628,20735
3,0011045010,5.691,-76.644,43970
4,0011050020,5.995,-76.780,21730
...,...,...,...,...
587,5202500132,1.814,-76.783,4878
588,5205500123,1.055,-77.270,12631
589,5311500056,3.848,-76.987,21572
590,5311500121,3.645,-76.565,25296


En la siguiente celda listamos los registros que presentan una distancia al punto medio mayor a 1.5Km. Realizamos para cada *CodSta* el cálculo de *%Entries*, esta última columna servirá como criterio para elegir una coordenada única para cada *CodSta*.

In [19]:
coordSta_3_temp = coordSta_1[coordSta_1['Distance']>1.5].copy()

coordSta_aux = coordSta_3_temp.groupby(['CodSta'], observed=True).agg({'TotalEntries':'sum'}).reset_index()
coordSta_3_temp = coordSta_3_temp.merge(coordSta_aux, how='inner', on='CodSta', suffixes=('','_byCodSta'))
coordSta_3_temp['%Entries'] = coordSta_3_temp['TotalEntries']/coordSta_3_temp['TotalEntries_byCodSta']
coordSta_3_temp = coordSta_3_temp.sort_values(['CodSta','%Entries'])
coordSta_3_temp = coordSta_3_temp.drop('TotalEntries_byCodSta', axis=1)
coordSta_3_temp.groupby('CodSta',observed=True).apply(lambda x: x, include_groups=False)

Lat    Long  TotalEntries  Lat_mean  Long_mean  Distance  \
CodSta                                                                       
0011175000 1    5.586 -76.650          9103     5.558    -76.642     3.237   
           0    5.529 -76.634         27207     5.558    -76.642     3.344   
0017015010 2   12.542 -81.731         63743    12.565    -81.716     3.032   
           3   12.588 -81.701        285484    12.565    -81.716     3.032   
0024015300 4    5.643 -73.518           183     5.650    -73.531     1.636   
           5    5.656 -73.544         30628     5.650    -73.531     1.586   
0024035410 6    5.726 -72.921           182     5.740    -72.916     1.652   
           7    5.753 -72.911        129194     5.740    -72.916     1.548   
0025025240 8    8.543 -74.627          9244     8.544    -74.585     4.620   
           9    8.544 -74.543         18240     8.544    -74.585     4.618   
0054025010 10   4.898 -76.677          8918     4.898    -76.454    24.706   
           11   4.898 -76.232         41832     4.898    -76.454    24.595   
2319500125 13   7.737 -73.049           179     7.694    -73.114     8.612   
           12   7.650 -73.178         20329     7.694    -73.114     8.584   
2401500086 15   5.554 -73.355           178     5.498    -73.454    12.603   
           14   5.442 -73.552         27281     5.498    -73.454    12.508   
2403500041 17   5.788 -73.052           183     5.788    -73.067     1.659   
           16   5.788 -73.082         27036     5.788    -73.067     1.659   
3206500045 18   3.444 -73.744           170     3.480    -73.742     4.009   
           19   3.516 -73.740         17759     3.480    -73.742     4.009   

               %Entries  
CodSta                   
0011175000 1   0.250702  
           0   0.749298  
0017015010 2   0.182526  
           3   0.817474  
0024015300 4   0.005939  
           5   0.994061  
0024035410 6   0.001407  
           7   0.998593  
0025025240 8   0.336341  
           9   0.663659  
0054025010 10  0.175724  
           11  0.824276  
2319500125 13  0.008728  
           12  0.991272  
2401500086 15  0.006482  
           14  0.993518  
2403500041 17  0.006723  
           16  0.993277  
3206500045 18  0.009482  
           19  0.990518

Teniendo en cuenta las observaciones, elegimos como criterio de selección *%Entries*>0.7, de manera que los registros de coordenadas anómalos serán reemplazados con los registros de coordenadas que cumplan esta condición.

In [20]:
coordSta_aux = coordSta_3_temp[coordSta_3_temp['%Entries']>0.7][['CodSta','Lat','Long']]
coordSta_3 = coordSta_3_temp.merge(coordSta_aux, how='left', on='CodSta', suffixes=('','_sel'))
coordSta_3.loc[coordSta_3['Lat_sel'].notna(),'Lat'] = coordSta_3['Lat_sel']
coordSta_3.loc[coordSta_3['Long_sel'].notna(),'Long'] = coordSta_3['Long_sel']

Los registros que cumplen la condición de tener un registro coordenado con *%Entries*>0.7 se muestran a continuación en el dataframe *coordSta4*:

In [21]:
coordSta_4 = coordSta_3[coordSta_3['Lat_sel'].notna()][coordSta_3.columns[:4]]
coordSta_aux = coordSta_4.groupby(['CodSta','Lat','Long'], observed=True).agg({'TotalEntries':'sum'}).reset_index()
coordSta_4 = coordSta_4.merge(coordSta_aux, how='left', on=['CodSta','Lat','Long'], suffixes=('','_adj'))
coordSta_4 = coordSta_4.drop('TotalEntries', axis=1)
coordSta_4.rename({'TotalEntries_adj':'TotalEntries'}, inplace=True, axis=1)
coordSta_4 = coordSta_4.drop_duplicates().reset_index(drop=True)
coordSta_4

,CodSta,Lat,Long,TotalEntries
0,0011175000,5.529,-76.634,36310
1,0017015010,12.588,-81.701,349227
2,0024015300,5.656,-73.544,30811
3,0024035410,5.753,-72.911,129376
4,0054025010,4.898,-76.232,50750
5,2319500125,7.650,-73.178,20508
6,2401500086,5.442,-73.552,27459
7,2403500041,5.788,-73.082,27219
8,3206500045,3.516,-73.740,17929


Por otra parte, a continuación se lista los registros que no cumplen dicha condición. Para estas estaciones realizamos una busqueda manual en GoogleEarth para identificar las coordenadas correctas de las estaciones.

In [22]:
coordSta_5 = coordSta_3[coordSta_3['Lat_sel'].isna()][coordSta_3.columns[:4]]
coordSta_aux = coordSta_5.merge(coordSta_0, how='left', on=['CodSta','Lat','Long','TotalEntries'])
coordSta_aux

,CodSta,Lat,Long,TotalEntries
0,0025025240,8.543,-74.627,9244
1,0025025240,8.544,-74.543,18240


Las coordenadas correctas de cada sitio son: 

    MAJAGUAL         '0025025240' : [8.543,-74.627]

In [23]:
coordSta_5 = coordSta_3[coordSta_3['Lat_sel'].isna()][coordSta_3.columns[:4]]
coordSta_aux = pd.DataFrame({'0025025240':[8.543,-74.627]
                             }, index=coordSta_5.columns[1:-1])
coordSta_aux = coordSta_aux.transpose().reset_index(names='CodSta')
coordSta_5 = coordSta_5.merge(coordSta_aux, how='left', on='CodSta', suffixes=('','_adj'))
coordSta_5['Lat'] = coordSta_5['Lat_adj']
coordSta_5['Long'] = coordSta_5['Long_adj']
coordSta_5 = coordSta_5.groupby(['CodSta','Lat','Long'],observed=True).agg({'TotalEntries':'sum'}).reset_index()
coordSta_5

,CodSta,Lat,Long,TotalEntries
0,0025025240,8.543,-74.627,27484


Finalmente, concatenamos los DataFrames que contienen las correcciones aplicadas en cada criterio usado:

    coordSta_2 : Radio de aproximación < 1.5Km
    coordSta_4 : Radio de aproximación > 1.5Km y registros con frecuencias relativas mayores a 0.7
    coordSta_5 : Corrección manual

Adicionalmente, realizamos una sumatoria del total de registros para cada CodSta con el fin de evaluar la integridad de la operación de limpieza. El valor inicial de registros es de *199781152*.

In [24]:
coordSta_f = pd.concat([coordSta_2,coordSta_4,coordSta_5], ignore_index=True)
print(f'Total Entries: {coordSta_f['TotalEntries'].agg('sum')}')
coordSta_f

Total Entries: 78017987


,CodSta,Lat,Long,TotalEntries
0,0011025501,5.889,-76.145,501365
1,0011030010,5.377,-76.612,40366
2,0011035030,5.285,-76.628,20735
3,0011045010,5.691,-76.644,43970
4,0011050020,5.995,-76.780,21730
...,...,...,...,...
597,2319500125,7.650,-73.178,20508
598,2401500086,5.442,-73.552,27459
599,2403500041,5.788,-73.082,27219
600,3206500045,3.516,-73.740,17929


#### Municipio, Departamento y ZonaHidrografica

Realizamos un conteo de los valores únicos de cada columna.

In [25]:
Ubic_0 = df_part[['CodSta','Dept','City']].copy()
Ubic_0 = Ubic_0.drop_duplicates().reset_index(drop=True)
Ubic_0.nunique()

CodSta    602
Dept       45
City      412
dtype: int64

##### Departamentos

Realizamos una limpieza sobre los nombres de los departamentos, esto lo realizamos de manera manual debido a que son correcciones ortográficas.

In [26]:
Dept = Ubic_0['Dept'].drop_duplicates().reset_index(drop=True)
Dept = Dept.astype('str')
print(list(Dept))

['CHOCO', 'CHOCÓ', 'ANTIOQUIA', 'CORDOBA', 'CÓRDOBA', 'SUCRE', 'BOLIVAR', 'BOLÍVAR', 'MAGDALENA', 'LA GUAJIRA', 'NORTE DE SANTANDER', 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA', 'ARCHIPIÉLAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA', 'SAN ANDRÉS PROVIDENCIA', 'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA', 'HUILA', 'CAUCA', 'TOLIMA', 'CUNDINAMARCA', 'BOGOTA D.C.', 'BOGOTA', 'BOGOTÁ', 'CALDAS', 'BOYACÁ', 'SANTANDER', 'CESAR', '<nil>', 'VALLE DEL CAUCA', 'RISARALDA', 'QUINDÍO', 'ATLANTICO', 'ATLÁNTICO', 'GUAINÍA', 'META', 'GUAVIARE', 'CASANARE', 'ARAUCA', 'VICHADA', 'VAUPÉS', 'PUTUMAYO', 'CAQUETA', 'CAQUETÁ', 'NARIÑO', 'AMAZONAS', 'NARINO']


In [27]:
Ubic_0.replace({'Dept':r'ATL.NTICO'},{'Dept':'ATLÁNTICO'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'.*BOG.*'},{'Dept':'BOGOTÁ, D.C.'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'BOL.VAR'},{'Dept':'BOLÍVAR'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'CAQUET.*'},{'Dept':'CAQUETÁ'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'CHOC.'},{'Dept':'CHOCÓ'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'C.RDOBA'},{'Dept':'CÓRDOBA'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'.*SAN AND.*'},{'Dept':'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA'},regex=True, inplace=True)
Ubic_0.replace({'Dept':'NARINO'},{'Dept':'NARIÑO'},regex=True, inplace=True)
Ubic_0.replace({'Dept':r'VAUP.*'},{'Dept':'VAUPÉS'},regex=True, inplace=True)
Dept = Ubic_0['Dept'].drop_duplicates().reset_index(drop=True)
Dept = Dept.astype('str')
print(list(Dept))


['CHOCÓ', 'ANTIOQUIA', 'CÓRDOBA', 'SUCRE', 'BOLÍVAR', 'MAGDALENA', 'LA GUAJIRA', 'NORTE DE SANTANDER', 'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA', 'HUILA', 'CAUCA', 'TOLIMA', 'CUNDINAMARCA', 'BOGOTÁ, D.C.', 'CALDAS', 'BOYACÁ', 'SANTANDER', 'CESAR', '<nil>', 'VALLE DEL CAUCA', 'RISARALDA', 'QUINDÍO', 'ATLÁNTICO', 'GUAINÍA', 'META', 'GUAVIARE', 'CASANARE', 'ARAUCA', 'VICHADA', 'VAUPÉS', 'PUTUMAYO', 'CAQUETÁ', 'NARIÑO', 'AMAZONAS']


Vemos en la siguiente celda que las columnas Dept, City y Zone contienen un valor nulo nil por tanto, vamos a obtener la información de las coordenadas asociadas a estos códigos de estación para recuperar los valores correctos de las columnas con estos datos nulos.

In [28]:
Ubic_D1 = Ubic_0[Ubic_0['Dept']=='<nil>']
Ubic_D1.merge(df_part, how='inner', on='CodSta')

,CodSta,Dept_x,City_x,Dept_y,City_y,Lat,Long,TotalEntries
0,0024037620,<nil>,<nil>,<nil>,<nil>,0.000,0.000,7233
1,0035237040,<nil>,<nil>,<nil>,<nil>,5.419,-71.666,74
2,0088112901,<nil>,<nil>,<nil>,<nil>,0.000,0.000,2520


Realizando una busquedo en google earth encontramos la siguiente información:

    {0035237040: {'Dept':'CASANARE','City':'TRINIDAD', 'Zone':'META'}}
    {0024037620: {'Dept':'BOYACÁ','City':'GUICÁN', 'Zone':'META'}}
    {0088112901: {'Dept':'BOGOTÁ, D.C.','City':'BOGOTÁ, D.C.', 'Zone':'MAGDALENA MEDIO'}}

In [35]:
Ubic_D2 =pd.DataFrame([
    {'CodSta':'0024037620','Dept':'BOYACÁ','City':'GUICÁN'},
    {'CodSta':'0035237040','Dept':'CASANARE','City':'TRINIDAD'},
    {'CodSta':'0088112901','Dept':'BOGOTÁ, D.C.','City':'BOGOTÁ, D.C.'}   
    ]
)
Ubic_D2

,CodSta,Dept,City
0,0024037620,BOYACÁ,GUICÁN
1,0035237040,CASANARE,TRINIDAD
2,0088112901,"BOGOTÁ, D.C.","BOGOTÁ, D.C."


In [36]:
Ubic_1 = Ubic_0.merge(Ubic_D2, how='left', on='CodSta', suffixes=('','_nil'))
Ubic_1.loc[Ubic_1['Dept_nil'].notna(),'Dept'] = Ubic_1['Dept_nil']
Ubic_1.loc[Ubic_1['City_nil'].notna(),'City'] = Ubic_1['City_nil']
Ubic_1 = Ubic_1[Ubic_1.columns[:-2]]
Ubic_1

,CodSta,Dept,City
0,0011025501,CHOCÓ,EL CARMEN
1,0011030010,CHOCÓ,CÉRTEGUI
2,0011030010,CHOCÓ,CÉRTEGUI
3,0011035030,CHOCÓ,UNIÓN PANAMERICANA
4,0011035030,CHOCÓ,UNIÓN PANAMERICANA
...,...,...,...
644,5202500132,CAUCA,SAN SEBASTIAN
645,5205500123,NARIÑO,PASTO
646,5311500056,VALLE DEL CAUCA,BUENAVENTURA
647,5311500121,VALLE DEL CAUCA,LA CUMBRE


In [39]:
print(Ubic_1['Dept'].unique())

['CHOCÓ' 'ANTIOQUIA' 'CÓRDOBA' 'SUCRE' 'BOLÍVAR' 'MAGDALENA' 'LA GUAJIRA'
 'NORTE DE SANTANDER'
 'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA' 'HUILA'
 'CAUCA' 'TOLIMA' 'CUNDINAMARCA' 'BOGOTÁ, D.C.' 'CALDAS' 'BOYACÁ'
 'SANTANDER' 'CESAR' 'VALLE DEL CAUCA' 'RISARALDA' 'QUINDÍO' 'ATLÁNTICO'
 'GUAINÍA' 'META' 'GUAVIARE' 'CASANARE' 'ARAUCA' 'VICHADA' 'VAUPÉS'
 'PUTUMAYO' 'CAQUETÁ' 'NARIÑO' 'AMAZONAS']


##### Municipios

Nos interesamos inicialmente en encontrar aquellas estaciones que tienen asociados más de un municipio.

In [40]:
Ubic_M1 = Ubic_1[['CodSta','City']].drop_duplicates()
Ubic_M1 = Ubic_M1[Ubic_M1['CodSta'].duplicated(keep=False)].reset_index(drop=True)
Ubic_M1 = Ubic_M1.merge(coordSta_f[['CodSta','Lat','Long']], how='inner', on='CodSta', suffixes=('','_adj'))
Ubic_M1 = Ubic_M1.sort_values(['CodSta','City']).reset_index(drop=True)
Ubic_M1 = Ubic_M1.groupby(['CodSta','Lat','Long'], observed=True)['City'].agg(list).reset_index()
Ubic_M1['Count'] = Ubic_M1['City'].apply(lambda x: len(x))
print(f'Maximum number of different cities by CodSta: {Ubic_M1['Count'].max()}')
Ubic_M1

Maximum number of different cities by CodSta: 3


,CodSta,Lat,Long,City,Count
0,0011175000,5.529,-76.634,"[CÉRTEGUI, EL ATRATO]",2
1,0017025020,13.360,-81.358,"[SAN ANDRES Y PROVIDENCIA, SAN ANDRES Y PROV...",2
2,0021205012,4.637,-74.089,"[BOGOTA, D.C, BOGOTÁ D.C]",2
3,0021205509,4.397,-74.628,"[BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]",3
4,0021205512,4.469,-74.079,"[BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]",3
5,0021205523,4.788,-74.050,"[BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]",3
6,0021205710,4.669,-74.103,"[BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]",3
7,0021206560,4.661,-74.135,"[BOGOTA D.C, BOGOTÁ D.C]",2
8,0021206600,4.782,-74.094,"[BOGOTA, D.C, BOGOTÁ D.C]",2
9,0021255090,5.002,-74.905,"[ARMERO, ARMERO (GUAYABAL)]",2


In [41]:
Ubic_M1['City'].drop_duplicates().reset_index(drop=True)

0                                 [CÉRTEGUI, EL ATRATO]
1     [SAN ANDRES Y  PROVIDENCIA, SAN ANDRES Y  PROV...
2                             [BOGOTA, D.C, BOGOTÁ D.C]
3                 [BOGOTA D.C, BOGOTA, D.C, BOGOTÁ D.C]
4                              [BOGOTA D.C, BOGOTÁ D.C]
5                           [ARMERO, ARMERO (GUAYABAL)]
6                                 [CHAPARRAL, PLANADAS]
7                                      [ACHÍ, MAJAGUAL]
8                               [ANSERMANUEVO, ARGELIA]
9                                    [TÁMESIS, VENECIA]
10                        [NÓVITA, SAN JOSÉ DEL PALMAR]
11               [ALTO BAUDÓ, ALTO BAUDÓ (PIE DE PATO)]
12                 [BAHÍA SOLANO, BAHÍA SOLANO (MUTIS)]
13                                      [SAMACÁ, TUNJA]
Name: City, dtype: object

Se podría realizar un reemplazo inicial en base a las siguientes condiciones:

    - Eliminar caracteres dentro de paréntesis
    - Hacer correcciones ortográficas sobre: 'San Andrés, providencia y santa catalina', 'Bogotá, D.C.', y 'Rio Iró' 

Los cambios los aplicaremos sobre el dataframe Ubic_1

In [42]:
Ubic_1.replace({'City':r'.\(.*\)'},{'City':''},regex=True, inplace=True)
Ubic_1.replace({'City':r'.*BOG.*'},{'City':'BOGOTÁ, D.C.'},regex=True, inplace=True)
Ubic_1.replace({'City':r'.*SAN AND.*'},{'City':'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA'},regex=True, inplace=True)
Ubic_1.replace({'City':r'RIO IR.*'},{'City':'RIO IRÓ'},regex=True, inplace=True)

De nuevo aplicamos el filtrado anterior para listar las estaciones que presentan mas de un municipio en los registros. Se evidencian 4 estaciones con este tipo de diferencias.

In [43]:
Ubic_M2 = Ubic_1[['CodSta','City']].drop_duplicates()
Ubic_M2 = Ubic_M2[Ubic_M2['CodSta'].duplicated(keep=False)].reset_index(drop=True)
Ubic_M2 = Ubic_M2.merge(coordSta_f[['CodSta','Lat','Long']], how='inner', on='CodSta', suffixes=('','_adj'))
Ubic_M2 = Ubic_M2.sort_values(['CodSta','City']).reset_index(drop=True)
Ubic_M2 = Ubic_M2.groupby(['CodSta','Lat','Long'], observed=True)['City'].agg(list).reset_index()
Ubic_M2['Count'] = Ubic_M2['City'].apply(lambda x: len(x))
print(f'Maximum number of different cities by CodSta: {Ubic_M2['Count'].max()}')
Ubic_M2

Maximum number of different cities by CodSta: 2


,CodSta,Lat,Long,City,Count
0,0011175000,5.529,-76.634,"[CÉRTEGUI, EL ATRATO]",2
1,0022045502,3.950,-75.667,"[CHAPARRAL, PLANADAS]",2
2,0025025240,8.543,-74.627,"[ACHÍ, MAJAGUAL]",2
3,0026115090,4.776,-76.144,"[ANSERMANUEVO, ARGELIA]",2
4,0026205501,5.626,-75.705,"[TÁMESIS, VENECIA]",2
5,0054025010,4.898,-76.232,"[NÓVITA, SAN JOSÉ DEL PALMAR]",2
6,2401500086,5.442,-73.552,"[SAMACÁ, TUNJA]",2


Para resolver este conflicto, realizamos una busqueda en Google Earth para encontrar el municipio relacionado con las coordenadas de la estación. Las correcciones son las siguientes:

    - 0011175000 EL ATRATO
    - 0022045502 PLANADAS
    - 0025025240 MAJAGUAL
    - 0026115090 ARGELIA
    - 0026205501 VENECIA
    - 0054025010 SAN JOSÉ DEL PALMAR
    - 2401500086 SAMACÁ


In [44]:
Ubic_1.loc[Ubic_1['CodSta']=='0011175000','City'] = 'EL ATRATO'
Ubic_1.loc[Ubic_1['CodSta']=='0022045502','City'] = 'PLANADAS'
Ubic_1.loc[Ubic_1['CodSta']=='0025025240','City'] = 'MAJAGUAL'
Ubic_1.loc[Ubic_1['CodSta']=='0026115090','City'] = 'ARGELIA'
Ubic_1.loc[Ubic_1['CodSta']=='0026205501','City'] = 'VENECIA'
Ubic_1.loc[Ubic_1['CodSta']=='0054025010','City'] = 'SAN JOSÉ DEL PALMAR'
Ubic_1.loc[Ubic_1['CodSta']=='2401500086','City'] = 'SAMACÁ'
Ubic_2=Ubic_1[['CodSta','City','Dept']].drop_duplicates().reset_index(drop=True)
Ubic_2

,CodSta,City,Dept
0,0011025501,EL CARMEN,CHOCÓ
1,0011030010,CÉRTEGUI,CHOCÓ
2,0011035030,UNIÓN PANAMERICANA,CHOCÓ
3,0011045010,QUIBDÓ,CHOCÓ
4,0011050020,MEDIO ATRATO,CHOCÓ
...,...,...,...
599,5202500132,SAN SEBASTIAN,CAUCA
600,5205500123,PASTO,NARIÑO
601,5311500056,BUENAVENTURA,VALLE DEL CAUCA
602,5311500121,LA CUMBRE,VALLE DEL CAUCA


Con las modificaciones realizadas se encuentra que cada estación ahora queda asociada a un municipio.

##### Estaciones registradas con diferentes departamentos

A continuación listamos un pequeño grupo de registros que presentan incoherencias en la columna Dept. Para corregir las incoherencias vamos a realizar una busqueda en internet para determinar el departamento correcto de cada municipio.

In [45]:
Ubic_2[Ubic_2['CodSta'].duplicated(keep=False)].sort_values('CodSta').reset_index(drop=True)

,CodSta,City,Dept
0,0025025240,MAJAGUAL,BOLÍVAR
1,0025025240,MAJAGUAL,SUCRE
2,2120500204,"BOGOTÁ, D.C.","BOGOTÁ, D.C."
3,2120500204,"BOGOTÁ, D.C.",CUNDINAMARCA


Los departamentos correctos de las estaciones son:

    - 0025025240 : SUCRE
    - 2120500204 : BOGOTÁ, D.C.

In [46]:
Ubic_2.loc[Ubic_2['CodSta']=='0025025240','Dept'] = 'SUCRE'
Ubic_2.loc[Ubic_2['CodSta']=='2120500204','Dept'] = 'BOGOTÁ, D.C.'
Ubic_f = Ubic_2.drop_duplicates().reset_index(drop=True)
Ubic_f

,CodSta,City,Dept
0,0011025501,EL CARMEN,CHOCÓ
1,0011030010,CÉRTEGUI,CHOCÓ
2,0011035030,UNIÓN PANAMERICANA,CHOCÓ
3,0011045010,QUIBDÓ,CHOCÓ
4,0011050020,MEDIO ATRATO,CHOCÓ
...,...,...,...
597,5202500132,SAN SEBASTIAN,CAUCA
598,5205500123,PASTO,NARIÑO
599,5311500056,BUENAVENTURA,VALLE DEL CAUCA
600,5311500121,LA CUMBRE,VALLE DEL CAUCA


In [47]:
coordSta_f

,CodSta,Lat,Long,TotalEntries
0,0011025501,5.889,-76.145,501365
1,0011030010,5.377,-76.612,40366
2,0011035030,5.285,-76.628,20735
3,0011045010,5.691,-76.644,43970
4,0011050020,5.995,-76.780,21730
...,...,...,...,...
597,2319500125,7.650,-73.178,20508
598,2401500086,5.442,-73.552,27459
599,2403500041,5.788,-73.082,27219
600,3206500045,3.516,-73.740,17929


Finalmente recopilamos los dos dataframes de limpieza de coordenadas y de ubicaciones de las 894 estaciones descritas en el dataset inicial. Adicionalmente, asignamos el tipo de dato category a las columnas: 'Dept','City'

In [48]:
dfSta = coordSta_f.merge(Ubic_f, how='inner', on='CodSta').sort_values(['Dept','City'])
dfSta[['Dept','City']] = dfSta[['Dept','City']].astype('category')
dfSta = dfSta.sort_values('CodSta').reset_index(drop=True)
dfSta['CodSta'] = dfSta['CodSta'].astype('uint64')
dfSta = dfSta.rename_axis('Sta').reset_index()
dfSta=dfSta[['Sta','CodSta','Dept','City','Lat','Long']]
dfSta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602 entries, 0 to 601
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   Sta     602 non-null    int64   
 1   CodSta  602 non-null    uint64  
 2   Dept    602 non-null    category
 3   City    602 non-null    category
 4   Lat     602 non-null    float64 
 5   Long    602 non-null    float64 
dtypes: category(2), float64(2), int64(1), uint64(1)
memory usage: 41.3 KB


In [49]:
dfSta

,Sta,CodSta,Dept,City,Lat,Long
0,0,11025501,CHOCÓ,EL CARMEN,5.889,-76.145
1,1,11030010,CHOCÓ,CÉRTEGUI,5.377,-76.612
2,2,11035030,CHOCÓ,UNIÓN PANAMERICANA,5.285,-76.628
3,3,11045010,CHOCÓ,QUIBDÓ,5.691,-76.644
4,4,11050020,CHOCÓ,MEDIO ATRATO,5.995,-76.780
...,...,...,...,...,...,...
597,597,5202500132,CAUCA,SAN SEBASTIAN,1.814,-76.783
598,598,5205500123,NARIÑO,PASTO,1.055,-77.270
599,599,5311500056,VALLE DEL CAUCA,BUENAVENTURA,3.848,-76.987
600,600,5311500121,VALLE DEL CAUCA,LA CUMBRE,3.645,-76.565


In [50]:
dfSta.to_parquet('../stations_t.parquet')

In [51]:
df_part=pd.read_parquet("../CleanDatasets2/presion.parquet")
df_part['FechaObservacion'] = pd.to_datetime(df_part['FechaObservacion'],format='%m/%d/%Y %I:%M:%S %p')

df_part['CodigoEstacion'] = df_part['CodigoEstacion'].astype(np.uint64)
df_part

,CodigoEstacion,CodigoSensor,FechaObservacion,ValorObservado,NombreEstacion,Departamento,Municipio,ZonaHidrografica,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,36015020,0255,2017-10-03 06:00:00,992.500000,EL DIAMANTE - AUT,CASANARE,PAZ DE ARIPORO,META,5.816194444,-71.41983333,Presión Atmosferica (1h),HPa
1,21195190,0255,2014-02-14 05:00:00,785.200012,PASCA - AUT,CUNDINAMARCA,PASCA,ALTO MAGDALENA,4.310111111,-74.31175,Presión Atmosferica (1h),HPa
2,21015050,0255,2013-09-27 18:00:00,805.299988,PURACE - AUT,HUILA,SAN AGUSTÍN,ALTO MAGDALENA,1.925916667,-76.42755556,Presión Atmosferica (1h),HPa
3,21115010,0255,2005-11-28 10:00:00,958.500000,DESIERTO TATACOA,HUILA,VILLAVIEJA,ALTO MAGDALENA,3.234,-75.168,Presión Atmosferica (1h),HPa
4,28035060,0255,2008-04-08 04:00:00,988.700012,FEDEARROZ - AUT,CESAR,VALLEDUPAR,CESAR,10.46361111,-73.24805556,Presión Atmosferica (1h),HPa
...,...,...,...,...,...,...,...,...,...,...,...,...
22611003,48015050,0258,2024-10-16 23:04:00,1000.500000,AEROPUERTO VASQUEZ COBO,AMAZONAS,LETICIA,AMAZONAS - DIRECTOS,-4.193861111,-69.94091667,GPRS - PRESIÓN ATMOSFÉRICA,hPA
22611004,48015050,0258,2024-10-16 09:08:00,1004.099976,AEROPUERTO VASQUEZ COBO,AMAZONAS,LETICIA,AMAZONAS - DIRECTOS,-4.193861111,-69.94091667,GPRS - PRESIÓN ATMOSFÉRICA,hPA
22611005,16015501,0258,2024-10-16 19:02:00,972.900024,AEROPUERTO CAMILO DAZA,NORTE DE SANTANDER,CÚCUTA,CATATUMBO,7.93028,-72.50917,GPRS - PRESIÓN ATMOSFÉRICA,hPA
22611006,48015040,0255,2024-10-16 22:40:00,997.400024,PUERTO NARIÑO,AMAZONAS,PUERTO NARIÑO,AMAZONAS - DIRECTOS,-3.780305556,-70.36263889,PRESIÓN ATMOSFÉRICA,hPa


In [52]:
df_part=df_part[['CodigoEstacion','FechaObservacion','ValorObservado']]
df_part = df_part.rename({'CodigoEstacion':'CodSta',
                            'FechaObservacion':'Date',
                            'ValorObservado':'Pressure'
                            }, 
                            axis=1)
df_part

,CodSta,Date,Pressure
0,36015020,2017-10-03 06:00:00,992.500000
1,21195190,2014-02-14 05:00:00,785.200012
2,21015050,2013-09-27 18:00:00,805.299988
3,21115010,2005-11-28 10:00:00,958.500000
4,28035060,2008-04-08 04:00:00,988.700012
...,...,...,...
22611003,48015050,2024-10-16 23:04:00,1000.500000
22611004,48015050,2024-10-16 09:08:00,1004.099976
22611005,16015501,2024-10-16 19:02:00,972.900024
22611006,48015040,2024-10-16 22:40:00,997.400024


In [53]:
df_t=df_part.merge(dfSta,on='CodSta',how='inner')
df_t

,CodSta,Date,Pressure,Sta,Dept,City,Lat,Long
0,36015020,2017-10-03 06:00:00,992.500000,446,CASANARE,PAZ DE ARIPORO,5.816,-71.420
1,21195190,2014-02-14 05:00:00,785.200012,105,CUNDINAMARCA,PASCA,4.310,-74.312
2,21015050,2013-09-27 18:00:00,805.299988,62,HUILA,SAN AGUSTÍN,1.926,-76.428
3,21115010,2005-11-28 10:00:00,958.500000,86,HUILA,VILLAVIEJA,3.234,-75.168
4,28035060,2008-04-08 04:00:00,988.700012,387,CESAR,VALLEDUPAR,10.464,-73.248
...,...,...,...,...,...,...,...,...
22607897,48015050,2024-10-16 23:04:00,1000.500000,466,AMAZONAS,LETICIA,-4.194,-69.941
22607898,48015050,2024-10-16 09:08:00,1004.099976,466,AMAZONAS,LETICIA,-4.194,-69.941
22607899,16015501,2024-10-16 19:02:00,972.900024,44,NORTE DE SANTANDER,CÚCUTA,7.931,-72.510
22607900,48015040,2024-10-16 22:40:00,997.400024,465,AMAZONAS,PUERTO NARIÑO,-3.780,-70.363


In [54]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22607902 entries, 0 to 22607901
Data columns (total 8 columns):
 #   Column    Dtype         
---  ------    -----         
 0   CodSta    uint64        
 1   Date      datetime64[ns]
 2   Pressure  float32       
 3   Sta       int64         
 4   Dept      category      
 5   City      category      
 6   Lat       float64       
 7   Long      float64       
dtypes: category(2), datetime64[ns](1), float32(1), float64(2), int64(1), uint64(1)
memory usage: 1013.4 MB


In [56]:
df_t.to_parquet('../temperature.parquet')